In [123]:
! pip install seqeval==0.0.12 #BERT-large-cased

In [124]:
! pip install pytorch-transformers

In [125]:
!pip install keras_preprocessing

In [126]:
import pandas as pd
import math
import numpy as np
from seqeval.metrics import f1_score
from seqeval.metrics import classification_report,accuracy_score,f1_score
import torch.nn.functional as F
import torch
import os
from tqdm import tqdm,trange
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras_preprocessing.sequence import pad_sequences
from pytorch_transformers import BertTokenizer, BertConfig
from pytorch_transformers import BertForTokenClassification, AdamW

In [127]:
df_train = pd.read_csv("/kaggle/input/jargon-detection/train.csv",sep=",",encoding="latin1")
df_train.head()

,token_id,token,label
0,cs/doc_1/0/0,A,O
1,cs/doc_1/0/1,SHAP,O
2,cs/doc_1/0/2,summary,O
3,cs/doc_1/0/3,plot,TERM
4,cs/doc_1/0/4,shows,O


In [128]:
df_val = pd.read_csv("/kaggle/input/jargon-detection/validation.csv",sep=",",encoding="latin1")
df_val.head()

,token_id,token,label
0,cs/doc_1867/0/0,Section,O
1,cs/doc_1867/0/1,~,O
2,cs/doc_1867/0/2,illustrates,O
3,cs/doc_1867/0/3,the,O
4,cs/doc_1867/0/4,proposed,O


In [140]:
df_test = pd.read_csv("/kaggle/input/jargon-detection/test_no_labels.csv",sep=",",encoding="latin1")
df_test.shape

(42358, 3)

In [130]:
df_val.label.unique()

array(['O', 'TERM'], dtype=object)

In [131]:
df_train[['m','n','x','y']] = df_train["token_id"].apply(lambda x: pd.Series(str(x).split("/")))

In [132]:
df_train.label.value_counts()

O       520286
TERM     54624
Name: label, dtype: int64

In [133]:
df_val[['m','n','x','y']] = df_val["token_id"].apply(lambda x: pd.Series(str(x).split("/")))

In [141]:
df_test[['m','n','x','y']] = df_test["token_id"].apply(lambda x: pd.Series(str(x).split("/")))

In [135]:
df_train = df_train.dropna() 

In [136]:
df_val = df_val.dropna()

In [142]:
len(df_test)

42358

In [144]:
df_train[['a','b']] = df_train["n"].apply(lambda x: pd.Series(str(x).split("_")))
df_train

,token_id,token,label,m,n,x,y,a,b
0,cs/doc_1/0/0,A,O,cs,doc_1,0,0,doc,1
1,cs/doc_1/0/1,SHAP,O,cs,doc_1,0,1,doc,1
2,cs/doc_1/0/2,summary,O,cs,doc_1,0,2,doc,1
3,cs/doc_1/0/3,plot,TERM,cs,doc_1,0,3,doc,1
4,cs/doc_1/0/4,shows,O,cs,doc_1,0,4,doc,1
...,...,...,...,...,...,...,...,...,...
574905,phys/doc_1866/5/20,from,O,phys,doc_1866,5,20,doc,1866
574906,phys/doc_1866/5/21,a,O,phys,doc_1866,5,21,doc,1866
574907,phys/doc_1866/5/22,commercial,O,phys,doc_1866,5,22,doc,1866
574908,phys/doc_1866/5/23,supercapacitor,TERM,phys,doc_1866,5,23,doc,1866


In [145]:
df_val[['a','b']] = df_val["n"].apply(lambda x: pd.Series(str(x).split("_")))
df_val

,token_id,token,label,m,n,x,y,a,b
0,cs/doc_1867/0/0,Section,O,cs,doc_1867,0,0,doc,1867
1,cs/doc_1867/0/1,~,O,cs,doc_1867,0,1,doc,1867
2,cs/doc_1867/0/2,illustrates,O,cs,doc_1867,0,2,doc,1867
3,cs/doc_1867/0/3,the,O,cs,doc_1867,0,3,doc,1867
4,cs/doc_1867/0/4,proposed,O,cs,doc_1867,0,4,doc,1867
...,...,...,...,...,...,...,...,...,...
37138,phys/doc_1976/10/28,vorticity,TERM,phys,doc_1976,10,28,doc,1976
37139,phys/doc_1976/10/29,in,O,phys,doc_1976,10,29,doc,1976
37140,phys/doc_1976/10/30,the,O,phys,doc_1976,10,30,doc,1976
37141,phys/doc_1976/10/31,wake,O,phys,doc_1976,10,31,doc,1976


In [146]:
df_test[['a','b']] = df_test["n"].apply(lambda x: pd.Series(str(x).split("_")))
df_test

,token_id,token,label,m,n,x,y,a,b
0,cs/doc_1977/0/0,Next,O,cs,doc_1977,0,0,doc,1977
1,cs/doc_1977/0/1,",",O,cs,doc_1977,0,1,doc,1977
2,cs/doc_1977/0/2,we,O,cs,doc_1977,0,2,doc,1977
3,cs/doc_1977/0/3,analyze,O,cs,doc_1977,0,3,doc,1977
4,cs/doc_1977/0/4,the,O,cs,doc_1977,0,4,doc,1977
...,...,...,...,...,...,...,...,...,...
42353,phys/doc_2100/10/20,also,O,phys,doc_2100,10,20,doc,2100
42354,phys/doc_2100/10/21,presented,O,phys,doc_2100,10,21,doc,2100
42355,phys/doc_2100/10/22,and,O,phys,doc_2100,10,22,doc,2100
42356,phys/doc_2100/10/23,discussed,O,phys,doc_2100,10,23,doc,2100


In [147]:
df_train = df_train.drop('y', axis=1)
df_train = df_train.drop('token_id', axis=1)
df_train = df_train.drop('n', axis=1)
df_train = df_train.drop('a', axis=1)

In [148]:
df_val = df_val.drop('y', axis=1)
df_val = df_val.drop('token_id', axis=1)
df_val = df_val.drop('n', axis=1)
df_val = df_val.drop('a', axis=1)

In [149]:
df_test = df_test.drop('y', axis=1)
df_test = df_test.drop('token_id', axis=1)
df_test = df_test.drop('n', axis=1)
df_test = df_test.drop('a', axis=1)

In [150]:
df_train["s"] =  df_train["b"] + df_train['x'].astype(str) 

In [151]:
df_val["s"] = df_train["b"] + df_val['x'].astype(str)

In [152]:
df_test["s"] = df_test["b"] + df_test['x'].astype(str) 

In [153]:
! pip install scikit-learn

In [154]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder() 

In [155]:
df_train['c']= label_encoder.fit_transform(df_train['m'])

In [156]:
#Encode labels in column 'species'.
df_val['c']= label_encoder.fit_transform(df_val['m'])
df_val['m'].unique()

array(['cs', 'econ', 'phys'], dtype=object)

In [157]:
# Encode labels in column 'species'.
df_test['c']= label_encoder.fit_transform(df_test['m'])
df_test['m'].unique()

array(['cs', 'econ', 'phys'], dtype=object)

In [158]:
df_test["s"] = df_test['c'].astype(str) + df_test["s"]

In [159]:
df_val["s"] = df_val['c'].astype(str) + df_train["s"]

In [160]:
df_train["s"] = df_train["c"].astype(str) + df_train["s"]

In [161]:
df_train = df_train.drop('x', axis=1)
df_train = df_train.drop('b', axis=1)
df_train = df_train.drop('c', axis=1)

In [162]:
df_val = df_val.drop('x', axis=1)
df_val = df_val.drop('b', axis=1)
df_val = df_val.drop('c', axis=1)

In [163]:
df_test = df_test.drop('x', axis=1)
df_test = df_test.drop('b', axis=1)
df_test = df_test.drop('c', axis=1)

In [164]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(t,m,l) for t,m,l in zip(s["token"].values.tolist(),
                                                           s["m"].values.tolist(),
                                                           s["label"].values.tolist())]
        self.grouped = self.data.groupby("s").apply(agg_func)
        self.sentences = [s for s in self.grouped]

In [165]:
getter_trn = SentenceGetter(df_train)
sentences_trn = [[s[0] for s in sent] for sent in getter_trn.sentences]


In [166]:
getter_val = SentenceGetter(df_val)
sentences_val = [[s[0] for s in sent] for sent in getter_val.sentences]


In [167]:
getter_tst = SentenceGetter(df_test)
sentences_tst = [[str(s[0]) for s in sent] for sent in getter_tst.sentences]

In [168]:
topic_trn = [[s[1] for s in sent] for sent in getter_trn.sentences]
labels_trn = [[s[2] for s in sent] for sent in getter_trn.sentences]

print (len(sentences_trn[0]))
print (topic_trn[0])
print(labels_trn[0])

15
['cs', 'cs', 'cs', 'cs', 'cs', 'cs', 'cs', 'cs', 'cs', 'cs', 'cs', 'cs', 'cs', 'cs', 'cs']
['O', 'O', 'O', 'TERM', 'O', 'O', 'O', 'TERM', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [169]:
topic_val = [[s[1] for s in sent] for sent in getter_val.sentences]
labels_val = [[s[2] for s in sent] for sent in getter_val.sentences]

print (len(sentences_val[2]))
print (topic_val[2])
print(labels_val[2])

17
['cs', 'cs', 'cs', 'cs', 'cs', 'cs', 'cs', 'cs', 'cs', 'cs', 'cs', 'cs', 'cs', 'cs', 'cs', 'cs', 'cs']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'TERM', 'O', 'O', 'O', 'O']


In [170]:
topic_tst = [[s[1] for s in sent] for sent in getter_tst.sentences]
labels_tst = [[s[2] for s in sent] for sent in getter_tst.sentences]

print (len(sentences_tst[0]))
print (topic_tst[0])
print(labels_tst[0])

13
['cs', 'cs', 'cs', 'cs', 'cs', 'cs', 'cs', 'cs', 'cs', 'cs', 'cs', 'cs', 'cs']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [171]:
tags_vals = list(set(df_train["label"].values))

In [172]:
tags_vals = set(tags_vals)
tags_vals

{'O', 'TERM'}

In [173]:
tag2idx={'O': 0,
         'TERM' : 1,
         }

In [174]:
tag2name={tag2idx[key] : key for key in tag2idx.keys()}

In [175]:
device = torch.device("cuda" )if torch.cuda.is_available() else "cpu"
n_gpu = torch.cuda.device_count()

In [176]:
device

device(type='cuda')

In [177]:
# Len of the sentence must be the same as the training model
# See model's 'max_position_embeddings' = 512
max_len  = 40
batch_num = 64
# load tokenizer, with manual file address or pretrained address

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [178]:
sen_tst = []
for i in sentences_tst :
    for j in i:
        sen_tst.append(str(j))

In [179]:
sen_trn = []
for i in sentences_trn :
    for j in i:
        sen_trn.append(str(j))
lab_trn = []
for i in labels_trn :
    for j in i:
        lab_trn.append(str(j))        

In [180]:
sen_val = []
for i in sentences_val :
    for j in i:
        sen_val.append(str(j))
lab_val = []
for i in labels_val :
    for j in i:
        lab_val.append(str(j))        
        

In [181]:
tokenized_texts_tst = [sen_tst[i:i+40] for i in range(0, len(sen_tst), 40)]
tokenized_texts_tst ;   

In [182]:
tokenized_texts_trn = [sen_trn[i:i+40] for i in range(0, len(sen_trn), 40)]
tokenized_texts_trn ;   
word_piece_labels_trn = [lab_trn[i:i+40] for i in range(0, len(lab_trn), 40)]
word_piece_labels_trn ;  

In [183]:
tokenized_texts_val = [sen_val[i:i+40] for i in range(0, len(sen_val), 40)]
tokenized_texts_val ;   
word_piece_labels_val = [lab_val[i:i+40] for i in range(0, len(lab_val), 40)]
word_piece_labels_val ;   

In [184]:
input_ids_trn = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts_trn],
                          maxlen=max_len, dtype="long", truncating="post", padding="post")
print(input_ids_trn[0])

[  100   100 12654  5436  3065  1996   100  3643  1997  2838  2005  3265
  2951  2685  1012   100  2122  4275  2024  4050  2641   100  4083 13792
  1010  2144  2027  2024  2200  7591  2000   100  1013   100  1999  1996
  2731  2275  1066  1010]


In [185]:
input_ids_val = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts_val],
                          maxlen=max_len, dtype="long", truncating="post", padding="post")
print(input_ids_val[0])

[  100  1066 24899  1996  3818  9939  6741 10077  4118  1012   100  1066
  3065  1996  2291  1010  1996  3223 14090  2946  2071  2022  2172  3760
  2084  3025  8107  1010  1998  2947  2009  2038  1037  3760 17698  2946
  1012   100  2023  3259]


In [186]:
input_ids_tst = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts_tst],
                          maxlen=max_len, dtype="long", truncating="post", padding="post")
print(input_ids_tst[0])

[  100  1010  2057 17908  1996 23032  6075  6532  1006   100   100  1007
  1012   100 23032  6075  6532  2003  1996  6075  1997  1037   100 23032
  2029  2003  2062  2084  1037   100  3643  1006   100  1010  3770   100
  1997  1996  4555  2193]


In [187]:
input_ids_tst[15]

array([ 2000, 11487,  1999,  1011, 16602,   100,  1012,   100,  6709,
        2008,  2053,  6028,  3594,   100,  2636,  2005,  1996,  1999,
        1011, 16602,   100, 10788,  1012,   100,  2817,  1996, 20611,
        1997,  1996,  2193,  1997,  2335,  6705,  3431,  1006,   100,
        1998,  4555,  6705,  2689])

In [188]:
tags_trn = pad_sequences([[tag2idx.get(l) for l in lab] for lab in word_piece_labels_trn],
                     maxlen=max_len, value=tag2idx["O"], padding="post",
                     dtype="long", truncating="post")
print(tags_trn[0])

[0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0]


In [190]:
tags_val = pad_sequences([[tag2idx.get(l) for l in lab] for lab in word_piece_labels_val],
                     maxlen=max_len, value=tag2idx["O"], padding="post",
                     dtype="long", truncating="post")
print(tags_val[0])

[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
 0 0 0]


In [191]:
# For fine tune of predict, with token mask is 1,pad token is 0
attention_masks_trn = [[int(i>0) for i in ii] for ii in input_ids_trn]
attention_masks_trn[0];

In [192]:
# For fine tune of predict, with token mask is 1,pad token is 0
attention_masks_val = [[int(i>0) for i in ii] for ii in input_ids_val]
attention_masks_val[0];

In [193]:
# For fine tune of predict, with token mask is 1,pad token is 0
attention_masks_tst = [[int(i>0) for i in ii] for ii in input_ids_tst]
attention_masks_tst[0];

In [194]:
# Since only one sentence, all the segment set to 0
segment_ids_trn = [[0] * len(input_id) for input_id in input_ids_trn]
segment_ids_trn[0];

In [195]:
# Since only one sentence, all the segment set to 0
segment_ids_val = [[0] * len(input_id) for input_id in input_ids_val]
segment_ids_val[0];

In [196]:
# Since only one sentence, all the segment set to 0
segment_ids_tst = [[0] * len(input_id) for input_id in input_ids_tst]
segment_ids_tst[0];

In [197]:
tr_inputs = torch.tensor(input_ids_trn)
val_inputs = torch.tensor(input_ids_val)
tst_inputs = torch.tensor(input_ids_tst)

tr_tags = torch.tensor(tags_trn)
val_tags = torch.tensor(tags_val)

tr_masks = torch.tensor(attention_masks_trn)
val_masks = torch.tensor(attention_masks_val)
tst_masks = torch.tensor(attention_masks_tst)

tr_segs = torch.tensor(segment_ids_trn)
val_segs = torch.tensor(segment_ids_val)
tst_segs = torch.tensor(segment_ids_tst)

In [198]:
# Only set token embedding, attention embedding, no segment embedding
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
# Drop last can make batch training better for the last one
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_num,drop_last=True)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=batch_num)

In [199]:
test_data = TensorDataset(tst_inputs, tst_masks)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_num)

In [200]:
model = BertForTokenClassification.from_pretrained("bert-base-uncased",num_labels=len(tag2idx))

In [201]:
model = model.to(device)

In [202]:
# Set epoch and grad max num
epochs = 5
max_grad_norm = 1.0

In [203]:
# True: fine tuning all the layers 
# False: only fine tuning the classifier layers
FULL_FINETUNING = True

In [204]:
# Cacluate train optimiazaion num
num_train_optimization_steps = int( math.ceil(len(tr_inputs) / batch_num) / 1) * epochs

In [205]:
if FULL_FINETUNING:
    # Fine tune model all layer parameters
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    # Only fine tune classifier parameters
    param_optimizer = list(model.classifier.named_parameters()) 
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
optimizer = AdamW(optimizer_grouped_parameters, lr=0.0001)

In [206]:
# TRAIN loop
model.train();

In [207]:
print("***** Running training *****")
print("  Num examples = %d"%(len(tr_inputs)))
print("  Batch size = %d"%(batch_num))
print("  Num steps = %d"%(num_train_optimization_steps))
for _ in trange(epochs,desc="Epoch"):
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        
        # forward pass
        outputs = model(b_input_ids, token_type_ids=None,
        attention_mask=b_input_mask, labels=b_labels)
        loss, scores = outputs[:2]
        if n_gpu>1:
            # When multi gpu, average it
            loss = loss.mean()
        
        # backward pass
        loss.backward()
        
        # track train loss
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
        
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        
        # update parameters
        optimizer.step()
        optimizer.zero_grad()
        
    # print train loss per epoch
    print("Train loss: {}".format(tr_loss/nb_tr_steps))

***** Running training *****
  Num examples = 14372
  Batch size = 64
  Num steps = 1125


Epoch:  20%|██        | 1/5 [01:04<04:16, 64.16s/it]

Train loss: 0.14401751833169588


Epoch:  40%|████      | 2/5 [02:08<03:12, 64.18s/it]

Train loss: 0.10402587062812277


...

Train loss: 0.08253884549984443


Epoch:  80%|████████  | 4/5 [04:16<01:04, 64.03s/it]

Train loss: 0.06150494860152581


Epoch: 100%|██████████| 5/5 [05:20<00:00, 64.05s/it]

Train loss: 0.04441069113922173


In [208]:
bert_out_address = 'models/bert_out_model/en09'
# Make dir if not exits
if not os.path.exists(bert_out_address):
    os.makedirs(bert_out_address)

In [209]:
model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self

In [210]:
# If we save using the predefined names, we can load using `from_pretrained`
output_model_file = os.path.join(bert_out_address, "pytorch_model.bin")
output_config_file = os.path.join(bert_out_address, "config.json")

# Save model into file
torch.save(model_to_save.state_dict(), output_model_file)
model_to_save.config.to_json_file(output_config_file)
tokenizer.save_vocabulary(bert_out_address)

('models/bert_out_model/en09/vocab.txt',)

In [211]:
model.eval();

In [212]:
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
y_true = []
y_pred = []

print("***** Running evaluation *****")
print("  Num examples ={}".format(len(val_inputs)))
print("  Batch size = {}".format(batch_num))
for step, batch in enumerate(valid_dataloader):
    batch = tuple(t.to(device) for t in batch)
    input_ids, input_mask, label_ids = batch
    
    with torch.no_grad():
        outputs = model(input_ids, token_type_ids=None,
        attention_mask=input_mask,)
        # For eval mode, the first result of outputs is logits
        logits = outputs[0] 
   
    logits = torch.argmax(F.log_softmax(logits,dim=2),dim=2)
    logits = logits.detach().cpu().numpy()
    
    label_ids = label_ids.to('cpu').numpy()
    
    
    # Only predict the real word, mark=0, will not calculate
    input_mask = input_mask.to('cpu').numpy()
    
    # Compare the valuable predict result
    for i,mask in enumerate(input_mask):
        # Real one
        temp_1 = []
        # Predict one
        temp_2 = []
        
        for j, m in enumerate(mask):
            # Mark=0, meaning its a pad word, dont compare
            if m:
#                 if tag2name[label_ids[i][j]] != "X" : # Exclude the X label
                temp_1.append(tag2name[label_ids[i][j]])
                temp_2.append(tag2name[logits[i][j]])
            else:
                break
        
            
        y_true.append(temp_1)
        y_pred.append(temp_2)

# Get acc , recall, F1 result report
report = classification_report(y_true, y_pred,digits=4)


        
print("***** Eval results *****")
print("\n%s"%(report))
print("f1 score: %f"%(f1_score(y_true, y_pred)))
print("Accuracy score: %f"%(accuracy_score(y_true, y_pred)))


***** Running evaluation *****
  Num examples =929
  Batch size = 64
***** Eval results *****

           precision    recall  f1-score   support

     TERM     0.7206    0.7395    0.7299      3655

micro avg     0.7206    0.7395    0.7299      3655
macro avg     0.7206    0.7395    0.7299      3655

f1 score: 0.729949
Accuracy score: 0.946314


In [213]:
# Model we trained before, the dir containing pytorch_model.bin and vocab.txt
save_model_address = 'models/bert_out_model/en09'

In [214]:
save_model = BertForTokenClassification.from_pretrained(save_model_address,num_labels=len(tag2idx))

In [215]:
# Get model predict result
with torch.no_grad():
        outputs = save_model(tst_inputs, token_type_ids=None,attention_mask=None,)
        # For eval mode, the first result of outputs is logits
        #logits = outputs[0] 

In [216]:
# Make logits into numpy type predict result
# The predict result contain each token's all tags predict result
predict_results = outputs[0].detach().cpu().numpy()

In [217]:
predict_results.shape

(1059, 40, 2)

In [ ]:
# with torch.no_grad():
#     outputs = save_model(tst_inputs, token_type_ids=None,
#         attention_mask=None)
#         # For eval mode, the first result of outputs is logits
# logits = outputs[0] 
   
# logits = torch.argmax(F.log_softmax(logits,dim=2),dim=2)
# logits = logits.detach().cpu().numpy()

In [ ]:

# for i,mask in enumerate(input_mask):
#         # Real one
#         temp_2 = []
        
#         for j, m in enumerate(mask):
#             # Mark=0, meaning its a pad word, dont compare
#             if m:
# #                 if tag2name[label_ids[i][j]] != "X" : # Exclude the X label
                
#                 temp_2.append(tag2name[logits[i][j]])
#             else:
#                 break
#         y_pred.append(temp_2)


In [218]:
from scipy.special import softmax

In [219]:
result_list = []
for pre_res in predict_results:
    result_arrays_soft =[]
    for i in pre_res:
        result_arrays_soft.append(softmax(i))
    result_list.append(np.argmax(result_arrays_soft,axis= -1))    

In [220]:
print(result_list[9])

[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0]


In [221]:
print(input_ids_tst[15])

[ 2000 11487  1999  1011 16602   100  1012   100  6709  2008  2053  6028
  3594   100  2636  2005  1996  1999  1011 16602   100 10788  1012   100
  2817  1996 20611  1997  1996  2193  1997  2335  6705  3431  1006   100
  1998  4555  6705  2689]


In [222]:
print(tst_masks[9])

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])


In [ ]:
# answer = []
# for j in y_pred: 
#     for i in j:  
#         answer.append(i)      

In [223]:
answer = []
for j in range(len(result_list)): 
    for i, mark in enumerate(tst_masks[j]):
        if mark>0:
            if result_list[j][i] == 1 :   
                answer.append('TERM')      
            else :   
                answer.append('O')

In [224]:
len(answer)

42358

In [225]:
submission_df = pd.read_csv("/kaggle/input/jargon-detection/test_no_labels.csv",sep=",",encoding="latin1")

In [226]:
submission_df['label'] = pd.DataFrame(answer)

In [227]:
submission_df = submission_df.drop(['token'], axis=1)

In [228]:
submission_df.to_csv("debika_final.csv", index=False)